In [ ]:
from pathlib import Path

import pingouin as pg
from scipy.stats import shapiro

%reload_ext autoreload
%autoreload 2

import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns

import acr

warnings.filterwarnings("ignore")

In [ ]:
import pubplots as pp

from acr.utils import NNXR_GRAY, NREM_RED, PAPER_FIGURE_ROOT, SOM_BLUE

style_path = "/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr/src/acr/plot_styles/acrvec_labels.mplstyle"


nbroot = os.path.join(PAPER_FIGURE_ROOT, "response_to_review", "NFTR")
Path(nbroot).mkdir(parents=True, exist_ok=True)

# FUNCTIONS

# Analysis

In [ ]:
all_subjects = acr.nor.get_all_subjects()

In [ ]:
sub_dfs = []
c = -1
times = {}
speeds = {}

In [ ]:
group = "sleep"
sleep_subjects = []
for subject in all_subjects:
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == group:
        sleep_subjects.append(subject)

vel_dfs = {}
for subject in sleep_subjects:
    vel = acr.dlc.load_nor_actigraphy(subject)
    vel_dfs[subject] = vel
# get the mean of the velocity df (i.e. across all nodes), then do a robust z-score, then threshold sleep at -(mean/std), per validation on ephys mice. Only select the time of interest
# (i.e. the recovery period) after doing this full pipeline.

for subject in sleep_subjects:
    times[subject] = {}
    (
        times[subject]["acqday_start"],
        times[subject]["recovery_start"],
        times[subject]["recovery_end"],
    ) = acr.dlc.load_sleep_recovery_info(
        vel_dfs[subject], subject, recovery_duration="1h", buffer="5min"
    )
    acq_start = times[subject]["acqday_start"]
    t1 = acq_start - pd.Timedelta("6h")
    t2 = acq_start + pd.Timedelta("2h")
    v = vel_dfs[subject]
    # v = v.ts(acqday_start-pd.Timedelta('24h'), acqday_start+pd.Timedelta('36h'))
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    speeds[subject] = spd
    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c
    print(subject, len(spd.ts(t1, acq_start)))
    if len(spd.ts(t1, acq_start)) == 0:
        continue
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, acq_start), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "mean": mean,
            "std": std,
            "group": group,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

In [ ]:
group = "sd"
sleep_subjects = []
for subject in all_subjects:
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == group:
        sleep_subjects.append(subject)

for subject in sleep_subjects:
    vel = acr.dlc.load_nor_actigraphy(subject)
    vel_dfs[subject] = vel
# get the mean of the velocity df (i.e. across all nodes), then do a robust z-score, then threshold sleep at -(mean/std), per validation on ephys mice. Only select the time of interest
# (i.e. the recovery period) after doing this full pipeline.

for subject in sleep_subjects:
    times[subject] = {}
    (
        times[subject]["acqday_start"],
        times[subject]["recovery_start"],
        times[subject]["recovery_end"],
    ) = acr.dlc.load_sleep_recovery_info(
        vel_dfs[subject], subject, recovery_duration="1h", buffer="5min"
    )
    acq_start = times[subject]["acqday_start"]
    t1 = acq_start - pd.Timedelta("6h")
    t2 = acq_start + pd.Timedelta("2h")
    v = vel_dfs[subject]
    # v = v.ts(acqday_start-pd.Timedelta('24h'), acqday_start+pd.Timedelta('36h'))
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    speeds[subject] = spd
    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c
    print(subject, len(spd.ts(t1, acq_start)))
    if len(spd.ts(t1, acq_start)) == 0:
        continue
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, acq_start), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "mean": mean,
            "std": std,
            "group": group,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

In [ ]:
group = "stim"
sleep_subjects = []
for subject in all_subjects:
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == group:
        sleep_subjects.append(subject)

for subject in sleep_subjects:
    vel = acr.dlc.load_nor_actigraphy(subject)
    vel_dfs[subject] = vel
# get the mean of the velocity df (i.e. across all nodes), then do a robust z-score, then threshold sleep at -(mean/std), per validation on ephys mice. Only select the time of interest
# (i.e. the recovery period) after doing this full pipeline.

for subject in sleep_subjects:
    times[subject] = {}
    (
        times[subject]["acqday_start"],
        times[subject]["recovery_start"],
        times[subject]["recovery_end"],
    ) = acr.dlc.load_sleep_recovery_info(
        vel_dfs[subject], subject, recovery_duration="1h", buffer="5min"
    )
    acq_start = times[subject]["acqday_start"]
    t1 = acq_start - pd.Timedelta("6h")
    t2 = acq_start + pd.Timedelta("2h")
    v = vel_dfs[subject]
    # v = v.ts(acqday_start-pd.Timedelta('24h'), acqday_start+pd.Timedelta('36h'))
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    speeds[subject] = spd
    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c
    print(subject, len(spd.ts(t1, acq_start)))
    if len(spd.ts(t1, acq_start)) == 0:
        continue
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, acq_start), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "mean": mean,
            "std": std,
            "group": group,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)
sdff = pd.concat(sub_dfs)

In [ ]:
sub_dfs = []
c = -1
for subject in all_subjects:
    sub_type = acr.nor.get_subject_type(subject)
    acq_start = times[subject]["acqday_start"]
    t1 = acq_start - pd.Timedelta("6h")
    t2 = acq_start + pd.Timedelta("2h")
    v = vel_dfs[subject]
    # v = v.ts(acqday_start-pd.Timedelta('24h'), acqday_start+pd.Timedelta('36h'))
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c
    print(subject, len(spd.ts(t1, acq_start)))
    if len(spd.ts(t1, acq_start)) == 0:
        continue
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, acq_start), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)
sdf6 = pd.concat(sub_dfs)
sdf6 = pl.from_pandas(sdf6)

In [ ]:
sub_dfs = []
c = -1
for subject in all_subjects:
    sub_type = acr.nor.get_subject_type(subject)
    acq_start = times[subject]["acqday_start"]
    t1 = acq_start - pd.Timedelta("1h")
    t2 = acq_start + pd.Timedelta("2h")
    v = vel_dfs[subject]
    # v = v.ts(acqday_start-pd.Timedelta('24h'), acqday_start+pd.Timedelta('36h'))
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c
    print(subject, len(spd.ts(t1, acq_start)))
    if len(spd.ts(t1, acq_start)) == 0:
        continue
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, acq_start), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)
sdf1 = pd.concat(sub_dfs)
sdf1 = pl.from_pandas(sdf1)

In [ ]:
fig_name = "sleep_fraction_pre_acq_1hr"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
sleep = sdf1.filter(pl.col("group") == "sleep")["thresh_sleep"].to_numpy()
sd = sdf1.filter(pl.col("group") == "sd")["thresh_sleep"].to_numpy()
stim = sdf1.filter(pl.col("group") == "stim")["thresh_sleep"].to_numpy()
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(2.0, 1.33))
    ax, box = acr.plots.add_boxplot(
        ax,
        sleep,
        positions=[0.4],
        widths=0.0225,
        color=NREM_RED,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sleep, x_pos=0.42, color=NREM_RED, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.5],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.52, color=NNXR_GRAY, alpha=0.9, s=35, zorder=203
    )

    ax, box3 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.6],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.62, color=SOM_BLUE, alpha=0.9, s=35, zorder=204
    )
    ax.set_xlim(0.35, 0.65)
    ax.set_xticks([0.4, 0.5, 0.6])
    ax.set_xticklabels(["Sleep", "SD", "SD + OFF Induction"])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

In [ ]:
# Normality Checks
from scipy.stats import shapiro

_, p = shapiro(sleep)
print(f"p-value for sleep: {p}")

_, p = shapiro(sd)
print(f"p-value for sd: {p}")

_, p = shapiro(stim)
print(f"p-value for stim: {p}")

homo = pg.homoscedasticity(dv="thresh_sleep", group="group", data=sdf1.to_pandas())
print(homo["pval"][0], homo["equal_var"][0])


anov = pg.welch_anova(data=sdf1.to_pandas(), dv="thresh_sleep", between="group")
acr.stats.write_stats_result(
    fig_name,
    "Welch_ANOVA",
    test_statistic=anov["F"][0],
    p_value=anov["p-unc"][0],
    effect_size_method="np2",
    effect_size=anov["np2"][0],
    review=True,
)
anov.head()

In [ ]:
fig_name = "sleep_fraction_pre_acq_6hr"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
sleep = sdf6.filter(pl.col("group") == "sleep")["thresh_sleep"].to_numpy()
sd = sdf6.filter(pl.col("group") == "sd")["thresh_sleep"].to_numpy()
stim = sdf6.filter(pl.col("group") == "stim")["thresh_sleep"].to_numpy()
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(2.0, 1.33))
    ax, box = acr.plots.add_boxplot(
        ax,
        sleep,
        positions=[0.4],
        widths=0.0225,
        color=NREM_RED,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sleep, x_pos=0.42, color=NREM_RED, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.5],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.52, color=NNXR_GRAY, alpha=0.9, s=35, zorder=203
    )

    ax, box3 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.6],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.62, color=SOM_BLUE, alpha=0.9, s=35, zorder=204
    )
    ax.set_xlim(0.35, 0.65)
    ax.set_xticks([0.4, 0.5, 0.6])
    ax.set_xticklabels(["Sleep", "SD", "SD + OFF Induction"])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

In [ ]:
# Normality Checks
from scipy.stats import shapiro

_, p = shapiro(sleep)
print(f"p-value for sleep: {p}")

_, p = shapiro(sd)
print(f"p-value for sd: {p}")

_, p = shapiro(stim)
print(f"p-value for stim: {p}")

homo = pg.homoscedasticity(dv="thresh_sleep", group="group", data=sdf6.to_pandas())
print(homo["pval"][0], homo["equal_var"][0])


anov = pg.anova(data=sdf6.to_pandas(), dv="thresh_sleep", between="group")
acr.stats.write_stats_result(
    fig_name,
    "ANOVA",
    test_statistic=anov["F"][0],
    p_value=anov["p-unc"][0],
    effect_size_method="np2",
    effect_size=anov["np2"][0],
    review=True,
)
anov.head()

# Correlation to Performance

In [ ]:
pub_utils = acr.utils.import_publication_functions(
    "/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py",
    "pub_utils",
)
import pub_utils as pu

data_agg = acr.utils.import_publication_functions(
    "/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py",
    "data_agg",
)
import data_agg as dag

In [ ]:
from acr.stats import fit_line_and_mean_ci

In [ ]:
perf = pl.read_csv("../src_dat/nor_results.csv")

In [ ]:
perf = perf.filter(pl.col("subject") != "NOR_27")

In [ ]:
perf = perf.sort("subject")

In [ ]:
sdf6 = sdf6.sort("subject")
sdf1 = sdf1.sort("subject")

In [ ]:
perf = perf.with_columns(
    pl.lit(sdf6["thresh_sleep"]).alias("sleep6"),
    pl.lit(sdf1["thresh_sleep"]).alias("sleep1"),
)

In [ ]:
fig_name = "sleep_fraction_pre-v-perf_1hr_scatter"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 3, figsize=pp.scale(4, 1.33))
    x = perf.filter(pl.col("type") == "sleep")["sleep1"].to_numpy()
    y = perf.filter(pl.col("type") == "sleep")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[0] = sns.scatterplot(x=x, y=y, ax=ax[0], color=NREM_RED, s=150)
    ax[0].plot(x_grid, y_hat, color=NREM_RED, linewidth=2)
    ax[0].fill_between(x_grid, lo, hi, color=NREM_RED, alpha=0.2)
    x = perf.filter(pl.col("type") == "sd")["sleep1"].to_numpy()
    y = perf.filter(pl.col("type") == "sd")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[1] = sns.scatterplot(x=x, y=y, ax=ax[1], color=NNXR_GRAY, s=150)
    ax[1].plot(x_grid, y_hat, color=NNXR_GRAY, linewidth=2)
    ax[1].fill_between(x_grid, lo, hi, color=NNXR_GRAY, alpha=0.2)
    x = perf.filter(pl.col("type") == "stim")["sleep1"].to_numpy()
    y = perf.filter(pl.col("type") == "stim")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[2] = sns.scatterplot(x=x, y=y, ax=ax[2], color=SOM_BLUE, s=150)
    ax[2].plot(x_grid, y_hat, color=SOM_BLUE, linewidth=2)
    ax[2].fill_between(x_grid, lo, hi, color=SOM_BLUE, alpha=0.2)
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
    plt.show()

In [ ]:
groups = ["sleep", "sd", "stim"]  # adjust names to match your 'type' column

rows = []
for g in groups:
    stat_name = f"spearman_corr_PreAcqSleep-v-Performance__1hr__{g}"
    sub = perf.filter(pl.col("type") == g)
    x = sub["sleep1"].to_numpy()
    y = sub["ratio"].to_numpy()
    source_data = pd.DataFrame({"x": x, "y": y, "subject": np.arange(len(x))})
    pu.write_source_data(source_data, stat_name)

    # Spearman correlation
    out = pg.corr(x, y, method="spearman")
    out["group"] = g
    rows.append(out)

res = pd.concat(rows, ignore_index=True)

# Holm correction across the 3 group-wise correlations
reject, p_holm = pg.multicomp(res["p-val"].values, method="holm")
res["p_holm"] = p_holm
res["reject_holm"] = reject

# Keep only what you’ll report
res[["group", "n", "r", "CI95%", "p-val", "p_holm", "reject_holm"]]

for g in groups:
    stat_name = f"spearman_corr_PreAcqSleep-v-Performance__1hr__{g}"
    p_holm = res.loc[res["group"] == g, "p_holm"].values[0]
    rho = res.loc[res["group"] == g, "r"].values[0]

    acr.stats.write_stats_result(
        stat_name,
        "Spearman",
        test_statistic=rho,
        p_value=p_holm,
        effect_size_method="spearman-rho",
        effect_size=rho,
        review=True,
    )

In [ ]:
fig_name = "sleep_fraction_pre-v-perf_6hr_scatter"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 3, figsize=pp.scale(4, 1.33))
    x = perf.filter(pl.col("type") == "sleep")["sleep6"].to_numpy()
    y = perf.filter(pl.col("type") == "sleep")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[0] = sns.scatterplot(x=x, y=y, ax=ax[0], color=NREM_RED, s=150)
    ax[0].plot(x_grid, y_hat, color=NREM_RED, linewidth=2)
    ax[0].fill_between(x_grid, lo, hi, color=NREM_RED, alpha=0.2)
    x = perf.filter(pl.col("type") == "sd")["sleep6"].to_numpy()
    y = perf.filter(pl.col("type") == "sd")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[1] = sns.scatterplot(x=x, y=y, ax=ax[1], color=NNXR_GRAY, s=150)
    ax[1].plot(x_grid, y_hat, color=NNXR_GRAY, linewidth=2)
    ax[1].fill_between(x_grid, lo, hi, color=NNXR_GRAY, alpha=0.2)
    x = perf.filter(pl.col("type") == "stim")["sleep6"].to_numpy()
    y = perf.filter(pl.col("type") == "stim")["ratio"].to_numpy()
    x_grid, y_hat, lo, hi = fit_line_and_mean_ci(
        x, y, ci=0.95, method="bootstrap", n_boot=2000, seed=0
    )
    ax[2] = sns.scatterplot(x=x, y=y, ax=ax[2], color=SOM_BLUE, s=150)
    ax[2].plot(x_grid, y_hat, color=SOM_BLUE, linewidth=2)
    ax[2].fill_between(x_grid, lo, hi, color=SOM_BLUE, alpha=0.2)
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
    plt.show()

In [ ]:
groups = ["sleep", "sd", "stim"]  # adjust names to match your 'type' column

rows = []
for g in groups:
    stat_name = f"spearman_corr_PreAcqSleep-v-Performance__6hr__{g}"
    sub = perf.filter(pl.col("type") == g)
    x = sub["sleep6"].to_numpy()
    y = sub["ratio"].to_numpy()
    source_data = pd.DataFrame({"x": x, "y": y, "subject": np.arange(len(x))})
    pu.write_source_data(source_data, stat_name)

    # Spearman correlation
    out = pg.corr(x, y, method="spearman")
    out["group"] = g
    rows.append(out)

res = pd.concat(rows, ignore_index=True)

# Holm correction across the 3 group-wise correlations
reject, p_holm = pg.multicomp(res["p-val"].values, method="holm")
res["p_holm"] = p_holm
res["reject_holm"] = reject

# Keep only what you’ll report
res[["group", "n", "r", "CI95%", "p-val", "p_holm", "reject_holm"]]

for g in groups:
    stat_name = f"spearman_corr_PreAcqSleep-v-Performance__6hr__{g}"
    p_holm = res.loc[res["group"] == g, "p_holm"].values[0]
    rho = res.loc[res["group"] == g, "r"].values[0]

    acr.stats.write_stats_result(
        stat_name,
        "Spearman",
        test_statistic=rho,
        p_value=p_holm,
        effect_size_method="spearman-rho",
        effect_size=rho,
        review=True,
    )

# Post-Manipulation

In [ ]:
times = {}
for subject in all_subjects:
    times[subject] = {}
    (
        times[subject]["acqday_start"],
        times[subject]["recovery_start"],
        times[subject]["recovery_end"],
    ) = acr.dlc.load_sleep_recovery_info(
        vel_dfs[subject], subject, recovery_duration="1h", buffer="0min"
    )

## 1 hr post-manipulation

In [ ]:
req_len = 3601
sub_dfs = []
for subject in times.keys():
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == "sleep":
        continue
    rec_start = times[subject]["recovery_start"]
    t1 = rec_start
    t2 = t1 + pd.Timedelta("1h")
    v = vel_dfs[subject]

    bl_t1 = rec_start - pd.Timedelta("24h")
    bl_t2 = bl_t1 + pd.Timedelta("1h")
    # if sub_type == 'sleep':
    #    t1 = t1+pd.Timedelta('1h')
    #    t2 = t2+pd.Timedelta('1h')
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    print(subject, len(spd.ts(t1, t2)))
    if len(spd.ts(t1, t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no recovery DATA")
        print(len(spd.ts(t1, t2)))
        continue

    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c

    # spdplot = spd.ts(t1-pd.Timedelta('72h'), t2+pd.Timedelta('72h'))
    # f, ax = plt.subplots(1, 1, figsize=(24, 10))
    # sns.lineplot(x='datetime', y='speed_robust_z', data=spdplot, ax=ax)
    # ax.set_title(f'{subject} {rec_start} | {sub_type}')
    # ax.axvspan(t1, t2, color='blue', alpha=0.2)
    # ax.axvspan(bl_t1, bl_t2, color='gray', alpha=0.2)
    # plt.show()

    if len(spd.ts(bl_t1, bl_t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no BASELINE data")
        print(len(spd.ts(bl_t1, bl_t2)))
        bl_sleep_pct = np.nan
    else:
        bl_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
            spd.ts(bl_t1, bl_t2), thresh_sleep, col="speed_robust_z"
        )
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, t2), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "bl_sleep": bl_sleep_pct,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

sdf1 = pd.concat(sub_dfs)
sdf1 = pl.from_pandas(sdf1)

In [ ]:
fig_name = "raw_sleep_fraction_post_1hr"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
sd = sdf1.filter(pl.col("group") == "sd")["thresh_sleep"].to_numpy()
stim = sdf1.filter(pl.col("group") == "stim")["thresh_sleep"].to_numpy()
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(1, 1.2))
    ax, box = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.4],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.42, color=NNXR_GRAY, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.5],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.52, color=SOM_BLUE, alpha=0.9, s=35, zorder=203
    )

    ax.set_xlim(0.35, 0.55)
    ax.set_xticks([0.4, 0.5])
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")


_, p = shapiro(sd)
print(f"p-value for sleep: {p}")

_, p = shapiro(stim)
print(f"p-value for sd: {p}")

hg = pg.compute_effsize(sd, stim, paired=False, eftype="hedges")

stat = pg.ttest(sd, stim, paired=False)


In [ ]:
hg

In [ ]:
stat

In [ ]:
print(f'Shape of sd: {sd.shape}, shape of stim: {stim.shape}')
stat_name = fig_name
acr.stats.write_stats_result(
    stat_name,
    "welches_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": sd,  # data
    "subject": np.arange(len(sd)),  # subject index
    "subject_type": "sd",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stim,  # data
    "subject": np.arange(len(stim)),  # subject index
    "subject_type": "off-induction",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
fig_name = "sleep_fraction_post_1hr_vs_bl_stim_only"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")

stimbl = sdf1.filter(pl.col("group") == "stim").sort("subject")["bl_sleep"].to_numpy()
stimpos = (
    sdf1.filter(pl.col("group") == "stim").sort("subject")["thresh_sleep"].to_numpy()
)

with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        stimbl,
        stimpos,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
    )
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
from scipy.stats import shapiro

_, p = shapiro(stimbl)
print(f"p-value for sleep: {p}")

_, p = shapiro(stimpos)
print(f"p-value for stimpos: {p}")

hg = pg.compute_effsize(stimbl, stimpos, paired=True, eftype="hedges")

stat = pg.ttest(stimbl, stimpos, paired=True)
stat_name = fig_name

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": stimbl,  # data
    "subject": np.arange(len(stimbl)),  # subject index
    "condition": "baseline",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stimpos,  # data
    "subject": np.arange(len(stimpos)),  # subject index
    "condition": "post-manipulation",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
req_len = 21601
sub_dfs = []
for subject in times.keys():
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == "sleep":
        continue
    rec_start = times[subject]["recovery_start"]
    t1 = rec_start
    t2 = t1 + pd.Timedelta("6h")
    v = vel_dfs[subject]

    bl_t1 = rec_start - pd.Timedelta("24h")
    bl_t2 = bl_t1 + pd.Timedelta("6h")
    # if sub_type == 'sleep':
    #    t1 = t1+pd.Timedelta('1h')
    #    t2 = t2+pd.Timedelta('1h')
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    print(subject, len(spd.ts(t1, t2)))
    if len(spd.ts(t1, t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no recovery DATA")
        print(len(spd.ts(t1, t2)))
        continue

    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c

    # spdplot = spd.ts(t1-pd.Timedelta('72h'), t2+pd.Timedelta('72h'))
    # f, ax = plt.subplots(1, 1, figsize=(24, 10))
    # sns.lineplot(x='datetime', y='speed_robust_z', data=spdplot, ax=ax)
    # ax.set_title(f'{subject} {rec_start} | {sub_type}')
    # ax.axvspan(t1, t2, color='blue', alpha=0.2)
    # ax.axvspan(bl_t1, bl_t2, color='gray', alpha=0.2)
    # plt.show()

    if len(spd.ts(bl_t1, bl_t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no BASELINE data")
        print(len(spd.ts(bl_t1, bl_t2)))
        bl_sleep_pct = np.nan
    else:
        bl_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
            spd.ts(bl_t1, bl_t2), thresh_sleep, col="speed_robust_z"
        )
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, t2), thresh_sleep, col="speed_robust_z"
    )
    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "thresh_sleep": thresh_sleep_pct,
            "bl_sleep": bl_sleep_pct,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

sdf1 = pd.concat(sub_dfs)
sdf1 = pl.from_pandas(sdf1)

In [ ]:
fig_name = "raw_sleep_fraction_post_6hr"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
sd = sdf1.filter(pl.col("group") == "sd")["thresh_sleep"].to_numpy()
stim = sdf1.filter(pl.col("group") == "stim")["thresh_sleep"].to_numpy()
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(1, 1.2))
    ax, box = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.4],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.42, color=NNXR_GRAY, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.5],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.52, color=SOM_BLUE, alpha=0.9, s=35, zorder=203
    )

    ax.set_xlim(0.35, 0.55)
    ax.set_xticks([0.4, 0.5])
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

_, p = shapiro(sd)
print(f"p-value for sleep: {p}")

_, p = shapiro(stim)
print(f"p-value for sd: {p}")

hg = pg.compute_effsize(sd, stim, paired=False, eftype="hedges")

stat = pg.ttest(sd, stim, paired=False)
stat_name = fig_name
acr.stats.write_stats_result(
    stat_name,
    "welches_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": sd,  # data
    "subject": np.arange(len(sd)),  # subject index
    "subject_type": "sd",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stim,  # data
    "subject": np.arange(len(stim)),  # subject index
    "subject_type": "off-induction",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
fig_name = "sleep_fraction_post_6hr_vs_bl_stim_only"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")

stimbl = sdf1.filter(pl.col("group") == "stim").sort("subject")["bl_sleep"].to_numpy()
stimpos = (
    sdf1.filter(pl.col("group") == "stim").sort("subject")["thresh_sleep"].to_numpy()
)

with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        stimbl,
        stimpos,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
    )
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

_, p = shapiro(stimbl)
print(f"p-value for sleep: {p}")

_, p = shapiro(stimpos)
print(f"p-value for stimpos: {p}")

hg = pg.compute_effsize(stimbl, stimpos, paired=True, eftype="hedges")

stat = pg.ttest(stimbl, stimpos, paired=True)
stat_name = fig_name

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": stimbl,  # data
    "subject": np.arange(len(stimbl)),  # subject index
    "condition": "baseline",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stimpos,  # data
    "subject": np.arange(len(stimpos)),  # subject index
    "condition": "post-manipulation",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
fig_name = "actigraphy_example_post--labelled_long"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
subject = "NOR_47"
sub_type = acr.nor.get_subject_type(subject)
rec_start = times[subject]["recovery_start"]
t1 = rec_start
t2 = t1 + pd.Timedelta("1h")
v = vel_dfs[subject]

bl_t1 = rec_start - pd.Timedelta("24h")
bl_t2 = bl_t1 + pd.Timedelta("1h")
plt1 = times[subject]["acqday_start"] - pd.Timedelta("24h")
plt2 = times[subject]["acqday_start"] + pd.Timedelta("12h")
# if sub_type == 'sleep':
#    t1 = t1+pd.Timedelta('1h')
#    t2 = t2+pd.Timedelta('1h')
spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
spd = acr.dlc.rob_z_col(spd, "speed")
print(subject, len(spd.ts(t1, t2)))

mean = spd["speed_robust_z"].mean()
std = spd["speed_robust_z"].std()
base = mean / std
thresh_sleep = base * c

with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(6.5, 1.0))
    spdplot = spd.ts(plt1, plt2)
    spdplot = spdplot.to_pandas()
    spdplot.loc[spdplot["speed_robust_z"].isna(), "speed_robust_z"] = np.nan

    # Detect gaps in datetime and insert NaN rows to break the line
    spdplot = spdplot.sort_values("datetime").reset_index(drop=True)
    time_diff = spdplot["datetime"].diff()
    gap_threshold = pd.Timedelta("2s")  # Adjust threshold as needed
    gap_indices = spdplot.index[time_diff > gap_threshold].tolist()

    # Insert NaN rows at gap locations to break the line
    for i, idx in enumerate(gap_indices):
        insert_idx = idx + i  # Adjust for previously inserted rows
        gap_row = pd.DataFrame({
            "datetime": [spdplot.loc[insert_idx - 1, "datetime"] + pd.Timedelta("1s")],
            "speed_robust_z": [np.nan],
        })
        spdplot = pd.concat([
            spdplot.iloc[:insert_idx],
            gap_row,
            spdplot.iloc[insert_idx:],
        ]).reset_index(drop=True)

    ax.plot(spdplot["datetime"], spdplot["speed_robust_z"], linewidth=0.8)

    ax.hlines(90, t1, t2, color="blue", alpha=0.7)
    ax.hlines(90, bl_t1, bl_t2, color="gray", alpha=0.7)

    ax.hlines(100, t1, t2 + pd.Timedelta("5h"), color="blue", alpha=0.7)
    ax.hlines(100, bl_t1, bl_t2 + pd.Timedelta("5h"), color="gray", alpha=0.7)

    bl1 = times[subject]["acqday_start"] - pd.Timedelta("24h")
    bl2 = bl1 + pd.Timedelta("12h")
    ax.hlines(110, bl1, bl2, color="red", alpha=0.7)

    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(plt1, plt2)
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

In [ ]:
fig_name = "actigraphy_example_post_long"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
subject = "NOR_47"
sub_type = acr.nor.get_subject_type(subject)
rec_start = times[subject]["recovery_start"]
t1 = rec_start
t2 = t1 + pd.Timedelta("1h")
v = vel_dfs[subject]

bl_t1 = rec_start - pd.Timedelta("24h")
bl_t2 = bl_t1 + pd.Timedelta("1h")
plt1 = times[subject]["acqday_start"] - pd.Timedelta("24h")
plt2 = times[subject]["acqday_start"] + pd.Timedelta("12h")
# if sub_type == 'sleep':
#    t1 = t1+pd.Timedelta('1h')
#    t2 = t2+pd.Timedelta('1h')
spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
spd = acr.dlc.rob_z_col(spd, "speed")
print(subject, len(spd.ts(t1, t2)))

mean = spd["speed_robust_z"].mean()
std = spd["speed_robust_z"].std()
base = mean / std
thresh_sleep = base * c

with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(6.5, 1.0))
    spdplot = spd.ts(plt1, plt2)
    spdplot = spdplot.to_pandas()
    spdplot.loc[spdplot["speed_robust_z"].isna(), "speed_robust_z"] = np.nan

    # Detect gaps in datetime and insert NaN rows to break the line
    spdplot = spdplot.sort_values("datetime").reset_index(drop=True)
    time_diff = spdplot["datetime"].diff()
    gap_threshold = pd.Timedelta("2s")  # Adjust threshold as needed
    gap_indices = spdplot.index[time_diff > gap_threshold].tolist()

    # Insert NaN rows at gap locations to break the line
    for i, idx in enumerate(gap_indices):
        insert_idx = idx + i  # Adjust for previously inserted rows
        gap_row = pd.DataFrame({
            "datetime": [spdplot.loc[insert_idx - 1, "datetime"] + pd.Timedelta("1s")],
            "speed_robust_z": [np.nan],
        })
        spdplot = pd.concat([
            spdplot.iloc[:insert_idx],
            gap_row,
            spdplot.iloc[insert_idx:],
        ]).reset_index(drop=True)

    ax.plot(spdplot["datetime"], spdplot["speed_robust_z"], linewidth=0.8)

    # ax.hlines(90, t1, t2, color='blue', alpha=0.7)
    # ax.hlines(90, bl_t1, bl_t2, color='gray', alpha=0.7)
    # ax.hlines(100, t1, t2+pd.Timedelta('5h'), color='blue', alpha=0.7)
    # ax.hlines(100, bl_t1, bl_t2+pd.Timedelta('5h'), color='gray', alpha=0.7)
    # bl1 = times[subject]['acqday_start'] - pd.Timedelta('24h')
    # bl2 = bl1 + pd.Timedelta('12h')
    # ax.hlines(110, bl1, bl2, color='red', alpha=0.7)

    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(plt1, plt2)
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")

In [ ]:
# pre schematic ----------
fig_name = "actigraphy_example_pre--labelled"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
subject = "NOR_47"
sub_type = acr.nor.get_subject_type(subject)
rec_start = times[subject]["acqday_start"]
t1 = rec_start
t2 = t1 + pd.Timedelta("1h")
v = vel_dfs[subject]

bl_t1 = rec_start - pd.Timedelta("1h")
bl_t2 = rec_start
plt1 = times[subject]["acqday_start"] - pd.Timedelta("12h")
plt2 = times[subject]["acqday_start"] + pd.Timedelta("12h")
# if sub_type == 'sleep':
#    t1 = t1+pd.Timedelta('1h')
#    t2 = t2+pd.Timedelta('1h')
spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
spd = acr.dlc.rob_z_col(spd, "speed")
print(subject, len(spd.ts(t1, t2)))

mean = spd["speed_robust_z"].mean()
std = spd["speed_robust_z"].std()
base = mean / std
thresh_sleep = base * c

with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(6.5, 1.0))
    spdplot = spd.ts(plt1, plt2)
    spdplot = spdplot.to_pandas()
    spdplot.loc[spdplot["speed_robust_z"].isna(), "speed_robust_z"] = np.nan

    # Detect gaps in datetime and insert NaN rows to break the line
    spdplot = spdplot.sort_values("datetime").reset_index(drop=True)
    time_diff = spdplot["datetime"].diff()
    gap_threshold = pd.Timedelta("2s")  # Adjust threshold as needed
    gap_indices = spdplot.index[time_diff > gap_threshold].tolist()

    # Insert NaN rows at gap locations to break the line
    for i, idx in enumerate(gap_indices):
        insert_idx = idx + i  # Adjust for previously inserted rows
        gap_row = pd.DataFrame({
            "datetime": [spdplot.loc[insert_idx - 1, "datetime"] + pd.Timedelta("1s")],
            "speed_robust_z": [np.nan],
        })
        spdplot = pd.concat([
            spdplot.iloc[:insert_idx],
            gap_row,
            spdplot.iloc[insert_idx:],
        ]).reset_index(drop=True)

    ax.plot(spdplot["datetime"], spdplot["speed_robust_z"], linewidth=0.8)

    ax.hlines(90, bl_t1, bl_t2, color='gray', alpha=0.7)
    ax.hlines(100, bl_t1-pd.Timedelta('5h'), bl_t2, color='red', alpha=0.7)
    ax.hlines(110, plt1, rec_start, color='green', alpha=0.7)

    ax.set_xlabel(None) 
    ax.set_ylabel(None)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(plt1, plt2)
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
    

In [ ]:
# pre schematic ----------
fig_name = "actigraphy_example_pre--labelled"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
subject = "NOR_47"
sub_type = acr.nor.get_subject_type(subject)
rec_start = times[subject]["acqday_start"]
t1 = rec_start
t2 = t1 + pd.Timedelta("1h")
v = vel_dfs[subject]

bl_t1 = rec_start - pd.Timedelta("1h")
bl_t2 = rec_start
plt1 = times[subject]["acqday_start"] - pd.Timedelta("12h")
plt2 = times[subject]["acqday_start"] + pd.Timedelta("12h")
# if sub_type == 'sleep':
#    t1 = t1+pd.Timedelta('1h')
#    t2 = t2+pd.Timedelta('1h')
spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
spd = acr.dlc.rob_z_col(spd, "speed")
print(subject, len(spd.ts(t1, t2)))

mean = spd["speed_robust_z"].mean()
std = spd["speed_robust_z"].std()
base = mean / std
thresh_sleep = base * c

with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(6.5, 1.0))
    spdplot = spd.ts(plt1, plt2)
    spdplot = spdplot.to_pandas()
    spdplot.loc[spdplot["speed_robust_z"].isna(), "speed_robust_z"] = np.nan

    # Detect gaps in datetime and insert NaN rows to break the line
    spdplot = spdplot.sort_values("datetime").reset_index(drop=True)
    time_diff = spdplot["datetime"].diff()
    gap_threshold = pd.Timedelta("2s")  # Adjust threshold as needed
    gap_indices = spdplot.index[time_diff > gap_threshold].tolist()

    # Insert NaN rows at gap locations to break the line
    for i, idx in enumerate(gap_indices):
        insert_idx = idx + i  # Adjust for previously inserted rows
        gap_row = pd.DataFrame({
            "datetime": [spdplot.loc[insert_idx - 1, "datetime"] + pd.Timedelta("1s")],
            "speed_robust_z": [np.nan],
        })
        spdplot = pd.concat([
            spdplot.iloc[:insert_idx],
            gap_row,
            spdplot.iloc[insert_idx:],
        ]).reset_index(drop=True)

    ax.plot(spdplot["datetime"], spdplot["speed_robust_z"], linewidth=0.8)

    ax.hlines(90, bl_t1, bl_t2, color='gray', alpha=0.7)
    ax.hlines(100, bl_t1-pd.Timedelta('5h'), bl_t2, color='red', alpha=0.7)
    ax.hlines(110, plt1, rec_start, color='green', alpha=0.7)

    ax.set_xlabel(None) 
    ax.set_ylabel(None)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(plt1, plt2)
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
    

In [ ]:
# pre schematic ----------
fig_name = "actigraphy_example_pre"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
subject = "NOR_47"
sub_type = acr.nor.get_subject_type(subject)
rec_start = times[subject]["acqday_start"]
t1 = rec_start
t2 = t1 + pd.Timedelta("1h")
v = vel_dfs[subject]

bl_t1 = rec_start - pd.Timedelta("1h")
bl_t2 = rec_start
plt1 = times[subject]["acqday_start"] - pd.Timedelta("12h")
plt2 = times[subject]["acqday_start"] + pd.Timedelta("12h")
# if sub_type == 'sleep':
#    t1 = t1+pd.Timedelta('1h')
#    t2 = t2+pd.Timedelta('1h')
spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
spd = acr.dlc.rob_z_col(spd, "speed")
print(subject, len(spd.ts(t1, t2)))

mean = spd["speed_robust_z"].mean()
std = spd["speed_robust_z"].std()
base = mean / std
thresh_sleep = base * c

with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(6.5, 1.0))
    spdplot = spd.ts(plt1, plt2)
    spdplot = spdplot.to_pandas()
    spdplot.loc[spdplot["speed_robust_z"].isna(), "speed_robust_z"] = np.nan

    # Detect gaps in datetime and insert NaN rows to break the line
    spdplot = spdplot.sort_values("datetime").reset_index(drop=True)
    time_diff = spdplot["datetime"].diff()
    gap_threshold = pd.Timedelta("2s")  # Adjust threshold as needed
    gap_indices = spdplot.index[time_diff > gap_threshold].tolist()

    # Insert NaN rows at gap locations to break the line
    for i, idx in enumerate(gap_indices):
        insert_idx = idx + i  # Adjust for previously inserted rows
        gap_row = pd.DataFrame({
            "datetime": [spdplot.loc[insert_idx - 1, "datetime"] + pd.Timedelta("1s")],
            "speed_robust_z": [np.nan],
        })
        spdplot = pd.concat([
            spdplot.iloc[:insert_idx],
            gap_row,
            spdplot.iloc[insert_idx:],
        ]).reset_index(drop=True)

    ax.plot(spdplot["datetime"], spdplot["speed_robust_z"], linewidth=0.8)

    #ax.hlines(90, bl_t1, bl_t2, color='gray', alpha=0.7)
    #ax.hlines(100, bl_t1-pd.Timedelta('5h'), bl_t2, color='red', alpha=0.7)
    #ax.hlines(110, plt1, rec_start, color='green', alpha=0.7)

    ax.set_xlabel(None) 
    ax.set_ylabel(None)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(plt1, plt2)
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")
    

# actual speed during wake

In [ ]:
req_len = 3601
sub_dfs = []
for subject in times.keys():
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == "sleep":
        continue
    rec_start = times[subject]["recovery_start"]
    t1 = rec_start
    t2 = t1 + pd.Timedelta("1h")
    v = vel_dfs[subject]

    bl_t1 = rec_start - pd.Timedelta("24h")
    bl_t2 = bl_t1 + pd.Timedelta("1h")
    # if sub_type == 'sleep':
    #    t1 = t1+pd.Timedelta('1h')
    #    t2 = t2+pd.Timedelta('1h')
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    print(subject, len(spd.ts(t1, t2)))
    if len(spd.ts(t1, t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no recovery DATA")
        print(len(spd.ts(t1, t2)))
        continue

    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c

    # spdplot = spd.ts(t1-pd.Timedelta('72h'), t2+pd.Timedelta('72h'))
    # f, ax = plt.subplots(1, 1, figsize=(24, 10))
    # sns.lineplot(x='datetime', y='speed_robust_z', data=spdplot, ax=ax)
    # ax.set_title(f'{subject} {rec_start} | {sub_type}')
    # ax.axvspan(t1, t2, color='blue', alpha=0.2)
    # ax.axvspan(bl_t1, bl_t2, color='gray', alpha=0.2)
    # plt.show()
    bl_spd = 0

    if len(spd.ts(bl_t1, bl_t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no BASELINE data")
        print(len(spd.ts(bl_t1, bl_t2)))
        bl_spd = np.nan
    else:
        bl_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
            spd.ts(bl_t1, bl_t2), thresh_sleep, col="speed_robust_z"
        )
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, t2), thresh_sleep, col="speed_robust_z"
    )
    spd = spd.with_columns(pl.lit(thresh_sleep).alias("thresh"))
    spd = spd.with_columns(
        pl
        .when(pl.col("speed_robust_z") > pl.col("thresh"))
        .then(pl.lit("Wake"))
        .otherwise(pl.lit("Sleep"))
        .alias("state")
    )
    if bl_spd == 0:
        bl_spd = spd.ts(bl_t1, bl_t2).filter(pl.col("state") == "Wake")["speed"].mean()
    post_spd = spd.ts(t1, t2).filter(pl.col("state") == "Wake")["speed"].mean()

    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "bl_spd": bl_spd,
            "post_spd": post_spd,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

sdf1 = pd.concat(sub_dfs)
sdf1 = pl.from_pandas(sdf1)

In [ ]:
stim = sdf1.filter(pl.col("group") == "stim")
spd_post = stim["post_spd"].to_numpy()
spd_bl = stim["bl_spd"].to_numpy()

spbl = spd_bl / spd_bl
stim = spd_post / spd_bl
with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        spbl,
        stim,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
        one_sided=True,
    )
    plt.show()

In [ ]:
sd = sdf1.drop_nulls("bl_spd").filter(pl.col("group") == "sd")
spd_post = sd["post_spd"].to_numpy()
spd_bl = sd["bl_spd"].to_numpy()

spbl = spd_bl / spd_bl
sd = spd_post / spd_bl
with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        spbl,
        sd,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
        one_sided=True,
    )
    plt.show()

In [ ]:
fig_name = "speed_rel_2_bl--sd_v_stim"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(1, 1.2))
    ax, box = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.4],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.42, color=NNXR_GRAY, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.5],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.52, color=SOM_BLUE, alpha=0.9, s=35, zorder=203
    )

    ax.set_xlim(0.35, 0.55)
    ax.set_xticks([0.4, 0.5])
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")


_, p = shapiro(sd)
print(f"p-value for sleep: {p}")

_, p = shapiro(stim)
print(f"p-value for sd: {p}")

hg = pg.compute_effsize(sd, stim, paired=False, eftype="hedges")

stat = pg.ttest(sd, stim, paired=False)
stat_name = fig_name
acr.stats.write_stats_result(
    stat_name,
    "welches_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": sd,  # data
    "subject": np.arange(len(sd)),  # subject index
    "subject_type": "sd",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stim,  # data
    "subject": np.arange(len(stim)),  # subject index
    "subject_type": "off-induction",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat


In [ ]:
# -- 6 hrs --

In [ ]:
req_len = 21601
sub_dfs = []
for subject in times.keys():
    sub_type = acr.nor.get_subject_type(subject)
    if sub_type == "sleep":
        continue
    rec_start = times[subject]["recovery_start"]
    t1 = rec_start
    t2 = t1 + pd.Timedelta("6h")
    v = vel_dfs[subject]

    bl_t1 = rec_start - pd.Timedelta("24h")
    bl_t2 = bl_t1 + pd.Timedelta("6h")
    # if sub_type == 'sleep':
    #    t1 = t1+pd.Timedelta('1h')
    #    t2 = t2+pd.Timedelta('1h')
    spd = v.group_by("datetime").agg(pl.col("speed").mean()).sort(["datetime"])
    spd = acr.dlc.rob_z_col(spd, "speed")
    print(subject, len(spd.ts(t1, t2)))
    if len(spd.ts(t1, t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no recovery DATA")
        print(len(spd.ts(t1, t2)))
        continue

    mean = spd["speed_robust_z"].mean()
    std = spd["speed_robust_z"].std()
    base = mean / std
    thresh_sleep = base * c

    # spdplot = spd.ts(t1-pd.Timedelta('72h'), t2+pd.Timedelta('72h'))
    # f, ax = plt.subplots(1, 1, figsize=(24, 10))
    # sns.lineplot(x='datetime', y='speed_robust_z', data=spdplot, ax=ax)
    # ax.set_title(f'{subject} {rec_start} | {sub_type}')
    # ax.axvspan(t1, t2, color='blue', alpha=0.2)
    # ax.axvspan(bl_t1, bl_t2, color='gray', alpha=0.2)
    # plt.show()
    bl_spd = 0

    if len(spd.ts(bl_t1, bl_t2)) != req_len:
        print(f"{subject} {rec_start} {sub_type} no BASELINE data")
        print(len(spd.ts(bl_t1, bl_t2)))
        bl_spd = np.nan
    else:
        bl_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
            spd.ts(bl_t1, bl_t2), thresh_sleep, col="speed_robust_z"
        )
    thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(
        spd.ts(t1, t2), thresh_sleep, col="speed_robust_z"
    )
    spd = spd.with_columns(pl.lit(thresh_sleep).alias("thresh"))
    spd = spd.with_columns(
        pl
        .when(pl.col("speed_robust_z") > pl.col("thresh"))
        .then(pl.lit("Wake"))
        .otherwise(pl.lit("Sleep"))
        .alias("state")
    )
    if bl_spd == 0:
        bl_spd = spd.ts(bl_t1, bl_t2).filter(pl.col("state") == "Wake")["speed"].mean()
    post_spd = spd.ts(t1, t2).filter(pl.col("state") == "Wake")["speed"].mean()

    sub_df = pd.DataFrame(
        {
            "subject": subject,
            "bl_spd": bl_spd,
            "post_spd": post_spd,
            "mean": mean,
            "std": std,
            "group": sub_type,
        },
        index=[0],
    )
    sub_dfs.append(sub_df)

sdf1 = pd.concat(sub_dfs)
sdf1 = pl.from_pandas(sdf1)

In [ ]:
stim = sdf1.filter(pl.col("group") == "stim")
spd_post = stim["post_spd"].to_numpy()
spd_bl = stim["bl_spd"].to_numpy()

spbl = spd_bl / spd_bl
stim = spd_post / spd_bl
with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        spbl,
        stim,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
        one_sided=True,
    )
    plt.show()

In [ ]:
sd = sdf1.drop_nulls("bl_spd").filter(pl.col("group") == "sd")
spd_post = sd["post_spd"].to_numpy()
spd_bl = sd["bl_spd"].to_numpy()

spbl = spd_bl / spd_bl
sd = spd_post / spd_bl
with pp.destination("figma", style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(
        spbl,
        sd,
        colors=[SOM_BLUE, SOM_BLUE],
        alphas=[0.5, 0.95],
        dot_size=70,
        fsize=pp.scale(1, 1.2),
        one_sided=True,
    )
    plt.show()

In [ ]:
fig_name = "speed_rel_2_bl--sd_v_stim__6hr"
fig_path = os.path.join(nbroot, f"{fig_name}.svg")
with pp.destination("figma", style=style_path):
    f, ax = plt.subplots(1, 1, figsize=pp.scale(1, 1.2))
    ax, box = acr.plots.add_boxplot(
        ax,
        sd,
        positions=[0.4],
        widths=0.0225,
        color=NNXR_GRAY,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, sd, x_pos=0.42, color=NNXR_GRAY, alpha=0.9, s=35, zorder=202
    )

    ax, box2 = acr.plots.add_boxplot(
        ax,
        stim,
        positions=[0.5],
        widths=0.0225,
        color=SOM_BLUE,
        means=True,
        mean_color="gold",
        mean_linewidth=3.5,
        mean_linestyle="--",
        mean_dashes=(1.2, 1.2),
        alpha=0.9,
    )
    ax = acr.plots.add_data_points(
        ax, stim, x_pos=0.52, color=SOM_BLUE, alpha=0.9, s=35, zorder=203
    )

    ax.set_xlim(0.35, 0.55)
    ax.set_xticks([0.4, 0.5])
    ax.set_xticklabels([None, None])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches="tight")


_, p = shapiro(sd)
print(f"p-value for sleep: {p}")

_, p = shapiro(stim)
print(f"p-value for sd: {p}")

hg = pg.compute_effsize(sd, stim, paired=False, eftype="hedges")

stat = pg.ttest(sd, stim, paired=False)
stat_name = fig_name
acr.stats.write_stats_result(
    stat_name,
    "welches_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)
src1 = pd.DataFrame({
    "sleep_frac": sd,  # data
    "subject": np.arange(len(sd)),  # subject index
    "subject_type": "sd",  # subject type
})
src2 = pd.DataFrame({
    "sleep_frac": stim,  # data
    "subject": np.arange(len(stim)),  # subject index
    "subject_type": "off-induction",  # subject type
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat